In [1]:
from fedn.utils.helpers.helpers import get_helper
from ultralytics import YOLO
import torch
import collections
import numpy as np

HELPER_MODULE = "numpyhelper"
helper = get_helper(HELPER_MODULE)

def get_best_device():
    """ Get the best device available.

    :return: The best device available.
    :rtype: str
    """
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
    return device

def compile_model():
    """Compile the YOLO model.

    :return: The compiled YOLO model.
    :rtype: torch.nn.Module
    """
    device = get_best_device()
    return YOLO("model.yaml").to(device)

def load_parameters(model_path):
    """Load model parameters from a file.

    :param model_path: The path to the model parameters file.
    :type model_path: str
    :return: The YOLO model with loaded parameters.
    :rtype: YOLO
    """
    parameters_np = helper.load(model_path)

    yolo_model = compile_model()
    torch_model = yolo_model.model.model

    keys = list(torch_model.state_dict().keys())
    if len(parameters_np) != len(keys):
        raise ValueError(f"Mismatch: {len(parameters_np)} parameters vs {len(keys)} model keys")

    state_dict = collections.OrderedDict({
        key: torch.tensor(val) for key, val in zip(keys, parameters_np)
    })
    torch_model.load_state_dict(state_dict, strict=False)

    yolo_model.ckpt = {"model": torch_model}
    return yolo_model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/srv/hops/anaconda/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model = load_parameters("weights/face_finder_best.npz")

WARNING ⚠️ no model scale passed. Assuming scale='n'.


In [3]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [9]:
params = {
    'data': '/hopsfs/Jupyter/yolov8-face/data/widerface.yaml',
    'epochs': 20,
    'batch': 32,
    'imgsz': 640,
    'device': 0,
    'resume': False,
    'workers': 0,
    'cache': "ram",
    'amp': True,
}

print(params)

{'data': '/hopsfs/Jupyter/yolov8-face/data/widerface.yaml', 'epochs': 20, 'batch': 32, 'imgsz': 640, 'device': 0, 'resume': False, 'workers': 0, 'cache': 'ram', 'amp': True}


In [ ]:
model.train(**params)

Ultralytics 8.3.181 🚀 Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/hopsfs/Jupyter/yolov8-face/data/widerface.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.3±0.6 ms, read: 7.2±1.7 MB/s, size: 101.9 KB)


train: Scanning /hopsfs/Jupyter/yolov8-face/data/widerface/train... 12880 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12880/12880 [06:44<00:00, 31.86it/s]


train: New cache created: /hopsfs/Jupyter/yolov8-face/data/widerface/train.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (10.3GB RAM): 100%|██████████| 12880/12880 [01:58<00:00, 108.62it/s]


val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 7.0±3.9 MB/s, size: 117.6 KB)


val: Scanning /hopsfs/Jupyter/yolov8-face/data/widerface/val... 3226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3226/3226 [01:41<00:00, 31.82it/s]


val: New cache created: /hopsfs/Jupyter/yolov8-face/data/widerface/val.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.3GB RAM):  17%|█▋        | 551/3226 [00:04<00:23, 115.55it/s]

In [ ]:
# cat /srv/hops/anaconda/.config/Ultralytics/settings.json
# {
#   "settings_version": "0.0.6",
#   "datasets_dir": "/hopsfs/Jupyter/scaleout/datasets",
#   "weights_dir": "weights",
#   "runs_dir": "runs",
#   "uuid": "46725eb23f0b74c61f1eec400508dee6b1ba06e1832e3a96dc940cdecaa1e9db",
#   "sync": true,
#   "api_key": "",
#   "openai_api_key": "",
#   "clearml": true,
#   "comet": true,
#   "dvc": true,
#   "hub": true,
#   "mlflow": true,
#   "neptune": true,
#   "raytune": true,
#   "tensorboard": false,
#   "wandb": false,
#   "vscode_msg": true,
#   "openvino_msg": true
# }